### Credit Card Fraud Detection: Harnessing the Power of Machine Learning in Snowflake ML

Credit card fraud is a significant concern in the digital age, where billions of transactions occur daily across the globe. Detecting fraudulent activities quickly and accurately is paramount to safeguarding both consumers and financial institutions. Machine learning (ML) has emerged as a powerful tool in the fight against credit card fraud, offering sophisticated methods to identify suspicious activities in real-time.

The Challenge of Fraud Detection
Credit card fraud can take various forms, including unauthorized transactions, account takeovers, and synthetic identity fraud. Traditional rule-based systems, which rely on predefined patterns and thresholds, often fall short in detecting new and evolving fraud schemes. These systems can also generate a high number of false positives, leading to unnecessary transaction declines and customer dissatisfaction.

In this demo we will be walking through a use case where the ML Capabilities of the Snowflake Data Cloud will be leveraged for investigating the past transactions.
Snowflake ML Classification function offers automated predictions and data insights using machine learning. All you need to provide is your data and it doesn't require time series as well. The Machine learning model enables users to sort data into different classes using patterns detected within the training data.

***This is a very simplified set of data, but it will hopefully show you how the core functionality works within Snowflake.***

The following cell performs a couple of actions: 

Create a file format named CSVFORMAT.Create a stage named CREDITCARD_TRANSACTIONSThe stage points to the external storage where the credit card transactions dataset is stored.Lists the contents of the CREDITCARD_TRANSACTIONS stage.

In [ ]:
# make sure that you add the following packages
# snowflake-ml-python

import streamlit as st
import pandas as pd
from snowflake.snowpark import Window
from snowflake.snowpark.functions import *

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"credit_card_fraud", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_tutorial":0, "source":"notebook"}}


from snowflake.snowpark import functions as F
from snowflake.ml.feature_store import (
FeatureStore,
FeatureView,
Entity,
CreationMode
)
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

In [ ]:
USE DATABASE CC_FINS_DB;
USE SCHEMA ANALYTICS;

CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1
    TYPE = 'CSV';

-- Create a stage with the csv format to stage the credit card transactions
-- Note that we're trying to keep it pretty simple in this example. Because of that, all fraud takes place in one location. If you would like a more realistic dataset, you will need to modify this .csv file and load it to another stage and read that file instead.

CREATE STAGE IF NOT EXISTS CREDITCARD_TRANSACTIONS
    FILE_FORMAT =  CSVFORMAT 
    URL = 's3://sfquickstarts/sfguide_credit_card_fraud_detection_using_snowflakeml/fraud_detection_dataset.csv';
    
    
--Upload the file manually in case of internal stage or in case of external stage point to file storage
-- Inspect the content of stage
LS @CREDITCARD_TRANSACTIONS;

In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file and write it to a table
# We can now infer schema from CSV files.
transactions_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@CREDITCARD_TRANSACTIONS")

transactions_df.write.mode('overwrite').save_as_table('CREDITCARD_TRANSACTIONS')

transactions_df.show()

 Generate various features related to user spending patterns. These features include weekly, monthly, and yearly spending, as well as transactional statistics that can be used for further analysis or machine learning tasks such as fraud detection.

In [ ]:
WITH 
weekly_spending AS (
    SELECT
        USER_ID,
        DATE_TRUNC('week',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')) AS week,
        SUM(TRANSACTION_AMOUNT) AS total_spent_wk
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID,
        DATE_TRUNC('week',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI'))
),
mean_weekly_spending AS (
    SELECT
        USER_ID,
        AVG(total_spent_wk) AS mean_weekly_spent
    FROM
        weekly_spending
    GROUP BY
        USER_ID
),
monthly_spending AS (
    SELECT
        USER_ID,
        DATE_TRUNC('month',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')) AS month,
        SUM(TRANSACTION_AMOUNT) AS total_spent
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID,
        DATE_TRUNC('month',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI'))
),
mean_monthly_spending AS (
    SELECT
        USER_ID,
        AVG(total_spent) AS mean_monthly_spent
    FROM
        monthly_spending
    GROUP BY
        USER_ID
),
yearly_spending AS (
    SELECT
        USER_ID,
        DATE_TRUNC('year',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')) AS month,
        SUM(TRANSACTION_AMOUNT) AS total_spent_yr
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID,
        DATE_TRUNC('year',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI'))
),
mean_yearly_spending AS (
    SELECT
        USER_ID,
        AVG(total_spent_yr) AS mean_yearly_spent
    FROM
        yearly_spending
    GROUP BY
        USER_ID
),
features AS (
    SELECT
        USER_ID,
        COUNT(TRANSACTION_ID) AS total_transactions,
        AVG(TRANSACTION_AMOUNT) AS avg_per_transaction_amount,
        STDDEV(TRANSACTION_AMOUNT) AS stddev_transaction_amount,
        COUNT(DISTINCT MERCHANT) AS num_unique_merchants,
        round(COUNT(TRANSACTION_ID) / COUNT(DISTINCT DATE_TRUNC('month',TO_TIMESTAMP(TRANSACTION_DATE,'MM/DD/YY HH24:MI')),0)) AS transactions_per_month,
        COUNT(CASE WHEN IS_FRAUD = 1 THEN 1 END) * 1.0 / COUNT(TRANSACTION_ID) AS fraud_rate
    FROM
        CREDITCARD_TRANSACTIONS
    GROUP BY
        USER_ID
)
SELECT
    DISTINCT cc.USER_ID,total_transactions,avg_per_transaction_amount,stddev_transaction_amount,num_unique_merchants,
    ROUND(mws.mean_weekly_spent,2) as mean_weekly_spent,
    ROUND(mms.mean_monthly_spent,2) as mean_monthly_spent,
    ROUND(mys.mean_yearly_spent,2) as mean_yearly_spent
FROM
    CREDITCARD_TRANSACTIONS cc
    JOIN features fs ON cc.USER_ID = fs.User_ID
    JOIN mean_monthly_spending mms ON cc.USER_ID = mms.User_ID
    JOIN mean_yearly_spending mys ON cc.USER_ID = mys.User_ID
    JOIN mean_weekly_spending mws ON cc.USER_ID = mws.User_ID;

In [ ]:
cust_spdf= Generate_Customer_Features.to_df()

cust_spdf.write.mode('overwrite').save_as_table('customer_features')
cust_sdf=session.sql("select * from customer_features")

cust_sdf.show()

Generate cumulative behavioral metrics for users based on their transaction data, such as cumulative clicks and cumulative logins per hour. It involves the use of window functions and joins to combine and transform data from the CREDITCARD_TRANSACTIONS table.

In [ ]:
WITH cumulative_behavior AS (
    SELECT
        USER_ID,
        SESSION_ID,
        TRANSACTION_DATE,
        SUM(CLICKS) OVER (PARTITION BY USER_ID ORDER BY TRANSACTION_DATE RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_clicks,
        SUM(LOGIN_PER_HOUR) OVER (PARTITION BY USER_ID ORDER BY TRANSACTION_DATE RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_logins_per_hour
    FROM CREDITCARD_TRANSACTIONS
),
unique_transactions AS (
    SELECT DISTINCT
        USER_ID,
        SESSION_ID,
        TRANSACTION_DATE,
        TRANSACTION_ID,
        TIME_ELAPSED,
        CLICKS,
        LOCATION,
        LATITUDE,
        LONGITUDE
    FROM CREDITCARD_TRANSACTIONS
)
SELECT
    ut.SESSION_ID,
    ut.TRANSACTION_DATE,
    ut.TIME_ELAPSED,
    ut.CLICKS,
    ut.TRANSACTION_ID,
    ut.LOCATION,
    ut.LATITUDE,
    ut.LONGITUDE,
    cb.cumulative_clicks,
    cb.cumulative_logins_per_hour
FROM
    unique_transactions ut
    JOIN cumulative_behavior cb ON ut.USER_ID = cb.USER_ID 
    AND ut.SESSION_ID = cb.SESSION_ID
    AND ut.TRANSACTION_DATE = cb.TRANSACTION_DATE
ORDER BY
    ut.TRANSACTION_ID;

In [ ]:
# Generating a DataFrame of transaction features, saving it as a table in Snowflake

trans_df=Generate_Transaction_Features.to_df()

trans_df.write.mode('overwrite').save_as_table('transaction_features')
trans_sdf=session.sql("select * from transaction_features")

trans_df.show()

An entity is an abstraction over a set of primary keys used for looking up feature data. An Entity represents a real-world "thing" that has data associated with it. Below cell registers an entity for Customer and Transaction in Feature Store

In [ ]:
FS1=FeatureStore(
session=session,
database="CC_FINS_DB",
    name="ANALYTICS",
    default_warehouse="CC_FINS_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST)

customer_entity = Entity(name="UserId", join_keys=["USER_ID"])
FS1.register_entity(customer_entity)

transaction_entity = Entity(name="transId", join_keys=["TRANSACTION_ID"])
FS1.register_entity(transaction_entity)

A feature view is a group of logically-related features that are refreshed on the same schedule. The FeatureView constructor accepts a Snowpark DataFrame that contains the feature generation logic. The provided DataFrame must contain the join_keys columns specified in the entities associated with the feature view. In this example we are using time-series data, so we will also specify the timestamp column name.

Below cell creates a feature view for the customer features

In [ ]:
managed_fv = FeatureView(
    name="Customer_Features",
    entities = [customer_entity],
    feature_df=cust_sdf,
    overwrite=True
)

registered_fv = FS1.register_feature_view(
    feature_view=managed_fv,
    version="V1"   
)  

managed_fv = FeatureView(
    name="Trans_Features",
    entities = [transaction_entity],
    feature_df=trans_sdf,
    overwrite=True
    
)

registered_fv = FS1.register_feature_view(
    feature_view=managed_fv,
    version="V1"
)

 This completes the setup for the Database objects and Feature Store Producer workflow. The data and the features which have been generated is available for the consumer with appropritate privileges. Time to head on to the next notebook!